# Give a model time to think

Our upcoming strategy involves guiding the model to independently devise solutions before hastily reaching a conclusion. Occasionally, we achieve superior outcomes by explicitly directing the models to engage in reasoning and formulating their solutions before drawing conclusions.

I would like to emphasize the importance of allowing the model sufficient time to actively process information before determining the correctness of an answer.

- 🎯 ...

In [1]:
from util import local_settings
from env_colors import TerminalTextColor as ttc
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, messages=None):
    if not messages:
        messages = [{"role": "user", "content": prompt}]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

sk-Yuu6ZbvcrmJ6aYp5IUQiT3BlbkFJMcylU
First LLM API example
✅ OpenAI Key loaded (sk-Yuu6ZbvcrmJ6aYp5IUQiT3BlbkFJMcylU...)


### Tactic 1:

#### Specify the steps required to complete a task

If a model is making reasoning errors by  rushing to an incorrect conclusion, you should try reframing the query to request a chain or series of relevant reasoning before the model provides its final answer.

Another way to think about this is that if you give a model a task that's too complex for it to do in a short amount of time or in a small number of words, it may make up a guess which is likely to be incorrect.

So, in these situations, you can instruct the model to think longer about a problem, which means it's spending more computational effort on  the task. 

In [7]:
text = f"""
In a charming village, siblings Jack and Jill set out on
a quest to fetch water from a hilltop
well. As they climbed, singing joyfully, misfortune
struck—Jack tripped on a stone and tumbled
down the hill, with Jill following suit.
Though slightly battered, the pair returned home to
comforting embraces. Despite the mishap,
their adventurous spirits remained undimmed, and they
continued exploring with delight.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple
backticks with 1 sentence.
2 - Translate the summary into Spanish.
3 - List each name in the Spanish summary.
4 - As the final topic of the output, generate a JSON object with the names in the Spanish summary that contains the following keys: spanish_summary, names, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)

print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1 - The text tells the story of siblings Jack and Jill who go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips and tumbles down the hill, with Jill following suit. Despite the mishap, they return home and continue exploring with delight.

2 - El texto cuenta la historia de los hermanos Jack y Jill que van en una búsqueda para traer agua de un pozo en la cima de una colina, pero encuentran desgracia cuando Jack tropieza y cae rodando por la colina, seguido por Jill. A pesar del percance, regresan a casa y continúan explorando con alegría.

3 - Jack, Jill

4 - {
  "spanish_summary": "El texto cuenta la historia de los hermanos Jack y Jill que van en una búsqueda para traer agua de un pozo en la cima de una colina, pero encuentran desgracia cuando Jack tropieza y cae rodando por la colina, seguido por Jill. A pesar del percance, regresan a casa y continúan explorando con alegría.",
  "names": ["Jack", "Jill"],
  "num_names": 

### Ask for a specific output

You can specify the desired result after outlining the task steps to guide a particular output and ensure its expected occurrence. In this scenario, the model had the opportunity to methodically organize its reasoning by executing the task step by step and fulfilling the output accurately.

`Task`

`Steps`

`Output Format`

In [9]:
prompt_2 = f"""
Your task is to perform the following actions:
1 - Summarize the following text delimited by
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>

Summary: <summary>

Translation: <summary translation>

Names: <list of names in Italian summary>

Output JSON: <json with summary and num_names>

Text: <{text}>
"""

response = get_completion(prompt_2)

print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill go on a quest to fetch water from a hilltop well, but encounter misfortune when Jack trips and tumbles down the hill, with Jill following suit, yet they remain undeterred and continue exploring.

Translation: Jack et Jill partent à la recherche d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche et dévale la colline, suivi de Jill, mais ils restent déterminés et continuent à explorer.

Names: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill partent à la recherche d'eau d'un puits au sommet d'une colline, mais rencontrent un malheur lorsque Jack trébuche et dévale la colline, suivi de Jill, mais ils restent déterminés et continuent à explorer.", "num_names": 2}


### Tactic 2: 

#### Instruct the model to work out its own solution before rushing to a conclusion

Sometimes, we get better results when explicitly instructing the models to explain their solutions before concluding. 

This is the same idea we were discussing, giving the model time to work things out before just saying if an answer is correct, in the same way a person would. Think first :)

So, in this prompt, we're asking the model to determine if the student's solution is correct.

In [10]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financial.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

response = get_completion(prompt)

print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost as a function of the number of square feet.


❌ Note that the student's solution is actually not correct.

We can fix this by instructing the model to work out its own solution first.

In [11]:
prompt = f"""
Your task is to determine if the student's solution
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution
and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financial.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

1. Land cost: $100 / square foot
The cost of land is calculated by multiplying the size of the installation (x) by the cost per square foot ($100):
Land cost = 100 * x

2. Solar panel cost: $250 / square foot
The cost of solar panels is calculated by multiplying the size of the installation (x) by the cost per square foot ($250):
Solar panel cost = 250 * x

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus an additional $10 per square foot. So the maintenance cost is calculated by adding the flat fee to the product of the size of the installation (x) and the cost per square foot ($10):
Maintenance cost = 100,000 + 10 * x

Total cost for the first year of operations:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = 100